In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import mysql.connector


matplotlib.use("TkAgg")

remote = mysql.connector.connect(
    host = "----",
    port = "3306",
    user = "root",
    password = "----",
    database = "----"
)

cur = remote.cursor()

In [62]:
url = "https://racingnews365.com/height-weight-f1-drivers"
page = urlopen(url)

soup = BeautifulSoup(page, "html.parser")

In [3]:
import re

In [4]:
name = 'Alex Albon'
h = "1.86m/ (6' 1'')"
w = "74kg/163lb"

print((' ').join(re.findall('[A-Z][a-z]+', name)))
print(re.findall('\d.\d+m',h)[0])
print(re.findall('\d+kg',w)[0])

Alex Albon
1.86m
74kg


In [5]:
d = "F1 drivers' height and weight 2025 season"
print(re.findall('\d+',d)[1])

2025


Let's get (driver's name, driver's height, driver's weight) info of each season. (2021~2025)

In [49]:
content_fields = soup.find_all(class_="content-fields")
for each_content in content_fields:
    panel_list = each_content.find_all(class_="panel")
    for panel in panel_list:
        season = panel.find(class_="panel__header__title content__heading--section")
        if "age" in season.get_text() or '2025' in season.get_text():
            continue
        else:
            this_season = re.findall('\d+', season.get_text())[0] if len(re.findall('\d+', season.get_text())[0]) == 4 else re.findall('\d+', season.get_text())[1]
            driver_list = panel.find_all(class_="panel__content")
            for driver_info in driver_list:
                each_driver = driver_info.find_all("td")                    
                for idx, each in enumerate(each_driver):
                        if idx %3 == 0:
                            each_info = []
                            each_info.append(this_season)
                            name = (' ').join(re.findall('[A-Z][a-z]+', each.get_text().strip())).strip()
                            each_info.append(name)
                                
                        if idx %3 == 1:   
                            height = re.findall('\d.\d+m',each.get_text().strip())[0]
                            height = int(float(height[:-1])*100)
                            each_info.append(height)
                            
                        if idx %3 == 2:  
                            weight = re.findall('\d+kg',each.get_text().strip())[0]    
                            weight = int(weight[:-2])               
                            each_info.append(weight)
                            sql = "insert into driver_info (season, name, height, weight) values (%s, %s, %s, %s)"
                            
                            cur.execute(sql, (each_info[0], each_info[1], each_info[2], each_info[3]))
                            remote.commit()

In [52]:
years = ['2024', '2023', '2022', '2021']

In [61]:
for year in years:
    print(f"======{year}Season=====")
    season = year
    url_for_driver_point = "https://www.formula1.com/en/results/"+year+"/drivers"
    page = urlopen(url_for_driver_point)

    soup2 = BeautifulSoup(page, 'html.parser')

    f1_data_table = soup2.find(class_="f1-table f1-table-with-data w-full")

    tr = f1_data_table.find_all("tr")
    for idx, each in enumerate(tr):
        if idx == 0:
            continue
        each_name = each.find(class_="underline underline-offset-normal decoration-1 decoration-greyLight hover:decoration-brand-primary")
        each_name = each_name["href"].split('/')[2].replace('-', ' ').split(' ')
        for idx in range(len(each_name)):
            each_name[idx] = each_name[idx][0].upper() + each_name[idx][1:]
        
        each_name = (' ').join(each_name)

        """ Handling exceptions for each Name"""

        if each_name == "Guanyu Zhou":
            each_name = "Zhou Guanyu"
        elif each_name == "Alexander Albon":
            each_name = "Alex Albon"
        elif each_name == "Kimi Raeikkoenen":
            each_name = "Kimi Raikkonen"

        point = int(re.findall('\d+', each.get_text())[1])
        point_sql = "update driver_info set points = %s where season = %s and name = %s"
        
        cur.execute(point_sql, (point, season, each_name))
        remote.commit()

======2024Season=====
======2023Season=====
======2022Season=====
======2021Season=====


Data collection is over.

In [1]:
remote.close()

NameError: name 'remote' is not defined